# import

In [3]:
import matplotlib.pyplot as plt
#plt.style.available

In [5]:
from pylab import *
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import pyfits


from matplotlib.ticker import NullFormatter  # useful for `logit` scale
from astropy.time import Time

from astropy.stats import LombScargle
#plt.style.use('seaborn-whitegrid')

%matplotlib inline
%config InlineBackend.figure_format='svg'
#%config InlineBackend.figure_format='retina'

## def

In [ ]:
def set_ax_tick(ax):
    ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on',)
    ax.xaxis.set_tick_params(which='minor', size=5, width=2, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=5, width=2, direction='in', right='on')

def set_ax_locator(ax,xma=1,xmi=0.2,yma=1,ymi=0.2):
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(xma))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(xmi))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(yma))
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(ymi))

# AB magnitude System

In [ ]:
This magnitude system is defined such that, when monochromatic flux 

f is measured in erg sec^-1 cm^-2 Hz^-1,

m(AB) = -2.5 log(f) - 48.60

mAB = mVega + ∆m

f= 10**((-48.6-mVega - ∆m)/2.5)

# wise

## Vega to AB Magnitudes


In [ ]:
(mAB = mVega + Δm)

In [ ]:
data_allwise_epoch=pd.read_csv('/Users/brettlv/Downloads/wise_cl/NGC1566_multi_epoch.csv')
data_neowise=pd.read_csv('/Users/brettlv/Downloads/wise_cl/NGC1566_neo.csv')
delta_m_wise=[2.699,3.339,5.174,6.620]
delta_m_neowise=[2.699,3.339,2.699,3.339,5.174,6.620]

|  |$\Delta m$|
|  ----  | ----  |
|W1|  2.699|
|W2|  3.339|
|W3|  5.174|
|W4|  6.620|

In [ ]:
fig, axs = plt.subplots(1, 1,figsize=(8,8))
axs.set_title('NGC1566_wise')

for i in range(4):
    name_mjd='mjd'
    name_band=data_allwise_epoch.columns[i*3+8]
    name_err=data_allwise_epoch.columns[i*3+9]
    print(name_mjd,name_band,name_err,delta_m_wise[i])
    data_band=data_allwise_epoch[data_allwise_epoch[name_mjd].notnull()]  
    #mag_wise_data=pd.DataFrame({'t':data_band[name_mjd],'flux':data_band[name_band],'err':data_band[name_err]},index=None,columns=['t','flux','err'])    
    dcf_wise_data=pd.DataFrame({'t':data_band[name_mjd],'flux':(-48.6-data_band[name_band]-delta_m_wise[i])/2.5,'err':data_band[name_err]/2.5},index=None,columns=['t','flux','err'])    
    axs.errorbar(dcf_wise_data['t'],dcf_wise_data['flux'],dcf_wise_data['err'],fmt='.',label=name_band+'multi_epoch')
    

data_neowise_select=data_neowise.loc[:, ['w1mpro', 'w1sigmpro',
         'w2mpro', 'w2sigmpro','w1mpro_allwise', 'w1sigmpro_allwise', 'w2mpro_allwise',
        'w2sigmpro_allwise', 'w3mpro_allwise', 'w3sigmpro_allwise',
        'w4mpro_allwise', 'w4sigmpro_allwise']]

for i in range(6):
    name_mjd='mjd'
    name_band=data_neowise_select.columns[i*2]
    name_err=data_neowise_select.columns[i*2+1]
    print(name_mjd,name_band,name_err,delta_m_neowise[i])
     
    #mag_neowise_data=pd.DataFrame({'t':data_neowise[name_mjd],'flux':data_neowise[name_band],'err':data_neowise[name_err]},index=None,columns=['t','flux','err'])   
    dcf_neowise_data=pd.DataFrame({'t':data_neowise[name_mjd],'flux':(-48.6-data_neowise[name_band]-delta_m_neowise[i])/2.5,'err':data_neowise[name_err]/2.5},index=None,columns=['t','flux','err'])
    axs.errorbar(dcf_neowise_data['t'],dcf_neowise_data['flux'],dcf_neowise_data['err'],fmt='.',label=name_band+'neo')
    
axs.legend(loc=[1.0,0.7])
#axs.invert_yaxis()
  
axs.set_ylabel('log_F')    
#axs.set_ylabel('magnitude(AB)')
axs.set_xlabel('mjd')
plt.tight_layout()
plt.savefig('/Users/brettlv/Downloads/NGC1566/wise_lc.png',dpi=400)

# Swift

# XRT-fit

In [ ]:
data_xrt_fit_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/xrt_fit_result/NGC1566_xrt_fit.txt",delimiter='\s+',header=0)  

In [ ]:
data_xrt_fit_NGC1566.head()

## F-gamma

In [ ]:
fig = plt.figure(figsize=(6, 6))
#fig.subplots_adjust(left=0.1, right=0.85, bottom=0.12, top=0.95)
ax=fig.add_subplot(1,1,1)


lower_error_f = data_xrt_fit_NGC1566['xflux_lo']
upper_error_f = data_xrt_fit_NGC1566['xflux_up']
lower_error_g = data_xrt_fit_NGC1566['gamma_lo']
upper_error_g = data_xrt_fit_NGC1566['gamma_up']
asymmetric_error_f = [lower_error_f, upper_error_f]
asymmetric_error_g = [lower_error_g, upper_error_g]
ax.semilogx()
ax.errorbar(data_xrt_fit_NGC1566['xflux'],data_xrt_fit_NGC1566['gamma'],xerr=asymmetric_error_f,yerr=asymmetric_error_g,fmt='.',label='F-gamma') 

ax.grid(alpha=0.1,which='major', linestyle='--', linewidth=1)

set_ax_tick(ax)
#set_ax_locator(ax,xma=1,xmi=0.2,yma=1,ymi=0.2)
ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.5))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.25))
    
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2

plt.xlabel(r'$F_\mathregular{1-10\,keV}\mathregular{(erg \cdot cm^{-2} s^{-1})}$')
plt.ylabel(r'$\Gamma$')
plt.ylim(0.3,2.7)
#plt.xlim(5e-13,2.e-10)

plt.subplots_adjust(hspace = 0.0)
fig.savefig('/Users/brettlv/Downloads/NGC1566/NGC1566_Gamma_F.png', 
            dpi=400, transparent=False, bbox_inches='tight')

## XRT-count

In [ ]:
data_xrt_pc_NGC1566 = pd.read_table("/Users/brettlv/Downloads/NGC1566/PC.qdp.txt",delimiter='\s+',header=0)  
data_xrt_wt_NGC1566 = pd.read_table("/Users/brettlv/Downloads/NGC1566/WT.qdp.txt",delimiter='\s+',header=0)  

fig, axs = plt.subplots(2, 1,figsize=(8,8))
axs[0].set_title('NGC1566_xrt_pc')
axs[1].set_title('NGC1566_xrt_wt')


name_band_pc='xrt_pc'
name_band_wt='xrt_wt'

lower_error_pc = -data_xrt_pc_NGC1566['Rateneg']
upper_error_pc = data_xrt_pc_NGC1566['Ratepos']
asymmetric_error_pc = [lower_error_pc, upper_error_pc]
axs[0].errorbar(data_xrt_pc_NGC1566['mjd'],data_xrt_pc_NGC1566['Rate'],yerr=asymmetric_error_pc,fmt='.',label=name_band_pc)


lower_error_wt = -data_xrt_wt_NGC1566['Rateneg']
upper_error_wt = data_xrt_wt_NGC1566['Ratepos']
asymmetric_error_wt = [lower_error_wt, upper_error_wt]
axs[1].errorbar(data_xrt_wt_NGC1566['mjd'],data_xrt_wt_NGC1566['Rate'],yerr=asymmetric_error_wt,fmt='.',label=name_band_wt)


timerangel,timeranger=(58000,59100)
axs[0].set_xlim(timerangel,timeranger)
axs[1].set_xlim(timerangel,timeranger)

axs[0].legend(loc=[1.0,0.7])
axs[1].legend(loc=[1.0,0.7])
  
axs[0].set_ylabel('rate')
axs[0].set_xlabel('mjd')
axs[1].set_ylabel('rate')
axs[1].set_xlabel('mjd')
  
plt.tight_layout()
plt.savefig('/Users/brettlv/Downloads/NGC1566/qdp_xrt_lc%d-%d.png'%(timerangel,timeranger),dpi=400)


# UVOT

In [ ]:
data_ubb_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_ubb_sum.csv",delimiter='|',header=0)  
data_um2_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_um2_sum.csv",delimiter='|',header=0)  


data_uuu_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_uuu_sum.csv",delimiter='|',header=0)  
data_uvv_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_uvv_sum.csv",delimiter='|',header=0)  

data_uw1_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_uw1_sum.csv",delimiter='|',header=0)  
data_uw2_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/wise_cl/fig/NGC1566_uw2_sum.csv",delimiter='|',header=0)  


In [ ]:
from astropy import constants
from astropy import units as u
UV_namelist=['v','b','u','uvw1','uvm2','uvw2']
UV_datalist=[data_uvv_NGC1566,data_ubb_NGC1566,data_uuu_NGC1566,data_uw1_NGC1566,data_um2_NGC1566,data_uw2_NGC1566  ]
UV_wavelengthlist=np.array([5468,4392,3465,2600,2246,1928])*u.AA#A for v,b u,w1,m2,w2
UV_widthlist=np.array([769,975,785,693,498,657])*u.AA#A for v,b u,w1,m2,w2

# https://www.mssl.ucl.ac.uk/www_astro/uvot/uvot_instrument/filterwheel/filterwheel.html

#UV_wavelengthlist=np.array([5469,4392,3465,2600,2246,1928])*u.AA#A for v,b u,w1,m2,w2
#


UV_freqlist=(constants.c/(UV_wavelengthlist)).to(u.Hz).value

In [ ]:
UV_wavelengthlist,UV_freqlist

In [ ]:
data_xrt_fit_NGC1566.columns

In [ ]:
data_ubb_NGC1566.columns

# ASAS-SN

In [6]:
data_ASAS_NGC1566 = pd.read_csv("/Users/brettlv/Downloads/NGC1566/AP36757554_ASASSN-V J042000.52-545616.7 (65.00215, -54.93797) .csv",header=0)  

In [7]:
data_ASAS_NGC1566['hmjd']= data_ASAS_NGC1566['hjd']- 2400000.5

In [8]:
data_ASAS_NGC1566.columns

Index(['hjd', 'camera', 'filter', 'mag', 'mag err', 'flux (mJy)', 'flux err',
       'hmjd'],
      dtype='object')

In [9]:
data_ASAS_NGC1566.sort_values('hmjd')

,hjd,camera,filter,mag,mag err,flux (mJy),flux err,hmjd
294,2.456830e+06,be,V,11.904,0.02,66.421,1.222,56829.45076
91,2.456834e+06,be,V,11.922,0.02,65.339,1.202,56833.43126
112,2.456839e+06,be,V,11.902,0.02,66.560,1.225,56838.43952
31,2.456847e+06,be,V,11.925,0.02,65.126,1.198,56846.41803
89,2.456849e+06,be,V,11.902,0.02,66.547,1.224,56848.40158
...,...,...,...,...,...,...,...,...
9,2.458378e+06,be,V,11.855,0.02,69.512,1.279,58377.23678
65,2.458383e+06,be,V,11.851,0.02,69.757,1.284,58382.35619
231,2.458384e+06,be,V,11.862,0.02,69.043,1.270,58383.22370
14,2.458385e+06,be,V,11.842,0.02,70.337,1.294,58384.22022


## jd_MJD_HJD

An alternative to JD is the Modified Julian Date (MJD), an abbreviated, 5-digit version of the Julian Date defined as:

    MJD = JD - 2400000.5

In [ ]:
from __future__ import print_function, division
from PyAstronomy import pyasl
import datetime

In [ ]:
jd=data_ASAS_NGC1566['hjd'][0]

mjd=jd-2400000.5

print(data_ASAS_NGC1566['hjd'][0],mjd)

print("year = %4d, month = %2d, day = %2d, hour = %5.3f" \
      % tuple(pyasl.daycnv(jd)))
print()

print("year = %4d, month = %2d, day = %2d, \
      hour = %2d, minute = %2d, seconds = %2d, microseconds = %6d" \
      % tuple(pyasl.daycnv(jd, mode='dtlist')))
      
print()
dt = pyasl.daycnv(jd, mode='dt')
print("Datetime object: ", dt)

In [ ]:
print("year = %4d, month = %2d, day = %2d, hour = %5.3f" \
      % tuple(pyasl.daycnv(jd)))
print()

# Convert calendar date to JD
dt = datetime.datetime(1968, 5, 23, 12)
print("Input date: ", dt)
print("Corresponding Julian date: ", pyasl.jdcnv(dt))
print("Corresponding reduced Julian date: ", pyasl.juldate(dt))
print("Corresponds to weekday: ", pyasl.weekday(dt))
print()

# Get current Julian date and convert to heliocentric date
cjd = pyasl.get_juldate()
print("Current Julian date: ", cjd)
print("Current modified Julian date: ", cjd-2400000.5)

#65.001750  -54.937806 NGC1566
ra=65.001750 
dec=-54.937806

print("Current (reduced) heliocentric Julian date (ra=%s deg, dec=%s deg): "%(ra,dec),'\n',\
      pyasl.helio_jd(cjd-2.4e6-0.5, ra, dec,),'\n',
      pyasl.helio_jd(cjd-2.4e6-0.5, ra, dec,TIME_DIFF=True),'second')

In [ ]:
from astropy import time, coordinates as coord, units as u
ip_NGC1566 = coord.SkyCoord(ra,dec,unit=(u.deg, u.deg), frame='icrs')

In [ ]:
ip_NGC1566 

## converted the same way to the V UVOT Swift system.

In [ ]:
ASAS_data=pd.DataFrame({'t':data_ASAS_NGC1566['hmjd'],
                            'flux':data_ASAS_NGC1566['flux (mJy)'],
                            'err':data_ASAS_NGC1566['flux err']},
                           index=None,columns=['t','flux','err'])   

ASAS_data=ASAS_data.sort_values('t')
ASAS_data=ASAS_data.reset_index(drop=True)

print(data_uvv_NGC1566.columns[3],data_uvv_NGC1566.columns[6],data_uvv_NGC1566.columns[7])

data_uv=pd.DataFrame({'t':data_uvv_NGC1566[data_uvv_NGC1566.columns[3]],
                          'flux':data_uvv_NGC1566[data_uvv_NGC1566.columns[6]],
                          'err':data_uvv_NGC1566[data_uvv_NGC1566.columns[7]]},
                         index=None,columns=['t','flux','err'])
data_uv=data_uv.sort_values('t')
data_uv=data_uv.reset_index(drop=True)

print(ASAS_data.columns,data_uv.columns)

dcf_data_uv

ASAS_data

In [ ]:
def get_simultaneous_data(ASAS_data,data_uv_select,timebin=1.0):
    for index,j in enumerate(data_uv_select['t']):
        mjd_near_i=np.argmin(abs(ASAS_data['t']-j))         
        mjd_near=ASAS_data['t'][mjd_near_i] 
        if abs(mjd_near-j)<timebin:
            data_uv_select.loc[index,'f_ASAS'] = ASAS_data.iloc[mjd_near_i]['flux']
            data_uv_select.loc[index,'t_ASAS'] = ASAS_data.iloc[mjd_near_i]['t']
            
    overlap_data_V=  data_uv_select[data_uv_select['f_ASAS']!=0]                 
    return overlap_data_V

data_uv_select=data_uv.copy()
data_uv_select['f_ASAS']=0
data_uv_select['t_ASAS']=0
overlap_data_V=get_simultaneous_data(ASAS_data,data_uv_select,timebin=1.0)

In [ ]:
overlap_data_V['delta_f']=overlap_data_V['f_ASAS']-overlap_data_V['flux']

In [ ]:
overlap_data_V['ratio_f']=overlap_data_V['f_ASAS']/overlap_data_V['flux']

In [ ]:
overlap_data_V

In [ ]:
np.average(overlap_data_V['f_ASAS']-overlap_data_V['flux'])

In [ ]:
from scipy.stats import spearmanr#
from scipy.stats.stats import pearsonr



In [ ]:
spearmanr(overlap_data_V['f_ASAS'],overlap_data_V['flux'])

In [ ]:
pearsonr(overlap_data_V['f_ASAS'],overlap_data_V['flux'])

In [ ]:
plt.scatter(overlap_data_V['f_ASAS'],overlap_data_V['flux'])

## NGC1566_V

In [ ]:
fig = plt.figure(figsize=(6, 6))
#fig.subplots_adjust(left=0.1, right=0.85, bottom=0.12, top=0.95)
ax=fig.add_subplot(1,1,1)
#ax.set_title('NGC1566_V')

offset=48.8

ax.errorbar(data_ASAS_NGC1566['hmjd'],data_ASAS_NGC1566['flux (mJy)']-offset,data_ASAS_NGC1566['flux err'],fmt='.',label='ASAS_V-%.1f'%offset) 
ax.errorbar(data_uvv_NGC1566[data_uvv_NGC1566.columns[3]],data_uvv_NGC1566[data_uvv_NGC1566.columns[6]],yerr=data_uvv_NGC1566[data_uvv_NGC1566.columns[7]],fmt='.',label='UVOT_V')

set_ax_tick(ax)

timerangel,timeranger=(53500,59500)
ax.set_xlim(timerangel,timeranger)
ax.xaxis.set_tick_params(which='major', size=10, width=2, rotation=45,)
ax.legend(loc=[1.0,0.7])
ax.set_ylabel('Flux(mJy)')
ax.set_xlabel('MJD')
fig.savefig('/Users/brettlv/Downloads/NGC1566/dcf_file/V-band_%s-%s.png'%(timerangel,timeranger),
            dpi=400,transparent=False,bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(6, 6))
#fig.subplots_adjust(left=0.1, right=0.85, bottom=0.12, top=0.95)
ax=fig.add_subplot(1,1,1)
#ax.set_title('NGC1566_V')

ax.errorbar(data_ASAS_NGC1566['hmjd'],data_ASAS_NGC1566['flux (mJy)']-offset,data_ASAS_NGC1566['flux err'],fmt='.',label='ASAS_V-%.1f'%offset) 
ax.errorbar(data_uvv_NGC1566[data_uvv_NGC1566.columns[3]],data_uvv_NGC1566[data_uvv_NGC1566.columns[6]],yerr=data_uvv_NGC1566[data_uvv_NGC1566.columns[7]],fmt='.',label='UVOT_V')

set_ax_tick(ax)

timerangel,timeranger=(58300,58400)
ax.set_xlim(timerangel,timeranger)
ax.xaxis.set_tick_params(which='major', size=10, width=2, rotation=45,)
ax.legend(loc=[1.0,0.7])
ax.set_ylabel('Flux(mJy)')
ax.set_xlabel('MJD')
fig.savefig('/Users/brettlv/Downloads/NGC1566/V-band_%s-%s.png'%(timerangel,timeranger),
            dpi=400,transparent=False,bbox_inches='tight')

# XRT-f-g-UVOT-V-wise to dcf

In [ ]:
f= 10**((-48.6-mVega - ∆m)/2.5)

In [ ]:
offset

In [ ]:
fig, axs = plt.subplots(5, 1,figsize=(24,40))

axs[0].set_title('NGC1566_XRT')
axs[1].set_title('NGC1566_gamma_in-X-ray')
axs[2].set_title('NGC1566_UVOT')
axs[3].set_title('NGC1566_ASAS_V')
axs[4].set_title('NGC1566_wise')

axs[3].errorbar(data_ASAS_NGC1566['hmjd'],data_ASAS_NGC1566['flux (mJy)'],data_ASAS_NGC1566['flux err'],fmt='.',label='ASAS-SN_Vband') 

dcf_ASAS_data=pd.DataFrame({'t':data_ASAS_NGC1566['hmjd'],'flux':data_ASAS_NGC1566['flux (mJy)']-offset,'err':data_ASAS_NGC1566['flux err']},index=None,columns=['t','flux','err'])    
dcf_ASAS_data=dcf_ASAS_data.sort_values('t')
dcf_ASAS_data.to_csv('/Users/brettlv/Downloads/NGC1566/dcf_file/dcf_ASAS_V_data.csv',index=False,header=False)



for i in range(4):
    name_mjd='mjd'
    name_band=data_allwise_epoch.columns[i*3+8]
    name_err=data_allwise_epoch.columns[i*3+9]
    print(name_mjd,name_band,name_err)
    data_band=data_allwise_epoch[data_allwise_epoch[name_mjd].notnull()]
    
    logflux=(-48.6-data_band[name_band]-delta_m_wise[i])/2.5
    logfluxerr= data_band[name_err]/2.5
    
    flux=10**(logflux)
    fluxerr=flux*abs(logfluxerr/logflux)
    
    
    #mag_wise_data=pd.DataFrame({'t':data_band[name_mjd],'flux':data_band[name_band],'err':data_band[name_err]},index=None,columns=['t','flux','err'])    
    dcf_wise_data=pd.DataFrame({'t':data_band[name_mjd],'flux':flux,'err':fluxerr},index=None,columns=['t','flux','err'])   
    axs[4].errorbar(dcf_wise_data['t'],dcf_wise_data['flux'],dcf_wise_data['err'],fmt='.',label=name_band+'multi_epoch')
    dcf_wise_data=dcf_wise_data.sort_values('t')
    dcf_wise_data.to_csv('/Users/brettlv/Downloads/NGC1566/dcf_file/dcf_wise_multi_%s_data.csv'%name_band,index=False,header=False)    
   
    
data_neowise_select=data_neowise.loc[:, ['w1mpro', 'w1sigmpro',
         'w2mpro', 'w2sigmpro','w1mpro_allwise', 'w1sigmpro_allwise', 'w2mpro_allwise',
        'w2sigmpro_allwise', 'w3mpro_allwise', 'w3sigmpro_allwise',
        'w4mpro_allwise', 'w4sigmpro_allwise']]

for i in range(6):
    name_mjd='mjd'
    name_band=data_neowise_select.columns[i*2]
    name_err=data_neowise_select.columns[i*2+1]
    print(name_mjd,name_band,name_err)
    
    logflux=(-48.6-data_neowise[name_band]-delta_m_neowise[i])/2.5
    logfluxerr=data_neowise[name_err]/2.5
    
    flux=10**(logflux)
    fluxerr=flux*abs(logfluxerr/logflux)
    
    #mag_neowise_data=pd.DataFrame({'t':data_neowise[name_mjd],'flux':data_neowise[name_band],'err':data_neowise[name_err]},index=None,columns=['t','flux','err'])   
    dcf_neowise_data=pd.DataFrame({'t':data_neowise[name_mjd],'flux':flux,'err':fluxerr},index=None,columns=['t','flux','err'])
    axs[4].errorbar(dcf_neowise_data['t'],dcf_neowise_data['flux'],dcf_neowise_data['err'],fmt='.',label=name_band+'_neo') 
    dcf_neowise_data=dcf_neowise_data.sort_values('t')
    dcf_neowise_data.to_csv('/Users/brettlv/Downloads/NGC1566/dcf_file/dcf_neowise_%s_data.csv'%name_band,index=False,header=False)
    
    

lower_error = data_xrt_fit_NGC1566['xflux_lo']
upper_error = data_xrt_fit_NGC1566['xflux_up']
asymmetric_error = [lower_error, upper_error]
axs[0].errorbar(data_xrt_fit_NGC1566['obs_time'],data_xrt_fit_NGC1566['xflux'],yerr=asymmetric_error,fmt='.',label='XRT')

dcf_data_xrt_fit_NGC1566=pd.DataFrame({'t':data_xrt_fit_NGC1566['obs_time'],'flux':data_xrt_fit_NGC1566['xflux'],'err':data_xrt_fit_NGC1566['xflux_lo']},index=None,columns=['t','flux','err'])
dcf_data_xrt_fit_NGC1566=dcf_data_xrt_fit_NGC1566.sort_values('t')
dcf_data_xrt_fit_NGC1566.to_csv('/Users/brettlv/Downloads/NGC1566/dcf_file/dcf_xrt_fit_NGC1566.csv',index=False,header=False)

dcf_data_xrt_fit_NGC1566_e12=pd.DataFrame({'t':data_xrt_fit_NGC1566['obs_time'],'flux':data_xrt_fit_NGC1566['xflux']*1.0e12,'err':data_xrt_fit_NGC1566['xflux_lo']*1.0e12},index=None,columns=['t','flux','err'])
dcf_data_xrt_fit_NGC1566_e12=dcf_data_xrt_fit_NGC1566_e12.sort_values('t')
dcf_data_xrt_fit_NGC1566_e12.to_csv('/Users/brettlv/Downloads/NGC1566/dcf_file/dcf_e12_xrt_fit_NGC1566.csv',index=False,header=False)



lower_error = data_xrt_fit_NGC1566['gamma_lo']
upper_error = data_xrt_fit_NGC1566['gamma_up']
asymmetric_error = [lower_error, upper_error]
axs[1].errorbar(data_xrt_fit_NGC1566['obs_time'],data_xrt_fit_NGC1566['gamma'],yerr=asymmetric_error,fmt='.',label='gamma')

for i,j in zip(UV_namelist,UV_datalist):
    print(j.columns[3],j.columns[6],j.columns[7])
    axs[2].errorbar(j[j.columns[3]],j[j.columns[6]],yerr=j[j.columns[7]],fmt='.',label=i)
    dcf_data_uv=pd.DataFrame({'t':j[j.columns[3]],'flux':j[j.columns[6]],'err':j[j.columns[7]]},index=None,columns=['t','flux','err'])
    dcf_data_uv=dcf_data_uv.sort_values('t')
    dcf_data_uv.to_csv('/Users/brettlv/Downloads/NGC1566/dcf_file/dcf_uvot-%s.csv'%i,index=False,header=False)

   
    
timerangel,timeranger=(54400,59500)
for i in range(5):
    axs[i].set_xlim(timerangel,timeranger)
    axs[i].legend(loc=[1.0,0.5])
    
#
axs[0].set_ylabel('flux')
axs[2].set_ylabel('mjy')
axs[3].set_ylabel('mjy')
axs[4].set_ylabel('log_flux')
#axs[4].invert_yaxis()
axs[0].set_xlabel('mjd')

plt.tight_layout()
plt.savefig('/Users/brettlv/Downloads/NGC1566/dcf_file/XRT-UVOT-ASASV_lc%d-%d.png'%(timerangel,timeranger),
            dpi=400,transparent=False,bbox_inches='tight')


## combinedV

In [ ]:
combined_V=pd.concat([dcf_ASAS_data,data_uv],ignore_index=True)
combined_V.to_csv('/Users/brettlv/Downloads/NGC1566/dcf_file/dcf_combined_V.csv',index=False,header=False)

## rebin_neowise

In [ ]:
rebin_neowise_w1=pd.read_csv('/Users/brettlv/Downloads/NGC1566/dcf_file/rebin_dcf_neowise_w1.csv',header=None)
rebin_neowise_w2=pd.read_csv('/Users/brettlv/Downloads/NGC1566/dcf_file/rebin_dcf_neowise_w2.csv',header=None)
rebin_neowise_w1.columns=['t','flux','err']
rebin_neowise_w2.columns=['t','flux','err']

In [ ]:
fig, axs = plt.subplots(1, 1,figsize=(8,5))

plt.errorbar(rebin_neowise_w1['t'],rebin_neowise_w1['flux'],
             rebin_neowise_w1['err'],fmt='.',marker='*',label='w1')
plt.errorbar(rebin_neowise_w2['t'],rebin_neowise_w2['flux'],
             rebin_neowise_w2['err'],fmt='.',marker='*',label='w2')
plt.legend()


for i in data_rebin_neowise_w1[0]:
    axs[0].vlines(i,min(dcf_data_xrt_fit_NGC1566['flux']),max(dcf_data_xrt_fit_NGC1566['flux']))
    
for i in data_rebin_neowise_w1[0]:
    axs[2].vlines(i,0,30)    
    
for i in data_rebin_neowise_w1[0]:
    axs[3].vlines(i,min(dcf_ASAS_data['flux']),max(dcf_ASAS_data['flux']))
    
for i in data_rebin_neowise_w1[0]:
    axs[4].vlines(i,-24.5,-23.5)

# dcf

In [ ]:
ls /Users/brettlv/Downloads/NGC1566/dcf_file/*.csv > /Users/brettlv/Downloads/NGC1566/dcf_file/ngc1566.txt

In [ ]:
dcf_lists=[]
with open ('/Users/brettlv/Downloads/NGC1566/dcf_file/ngc1566.txt','r') as f:
    dcf_list=f.readlines()
    for line in dcf_list:
        dcf_lists.append(line.rstrip('\n')) 

In [ ]:
dcf_lists

In [ ]:
for i,j in enumerate(dcf_lists):
    print(i,j.split('/')[-1][4:-4])

## def pydcf

In [ ]:
def get_data(path):    
    data=pd.read_csv(path,header=None)
    return data

def get_min_max(data):
    return min(data[0]),max(data[1])

def get_lc_data(data):    
    t1,dy1,y_obs1= (data[0],data[2],data[1]) 
    return t1,dy1,y_obs1

def get_new_range_data(t,dy,y_obs,rangemin,rangemax):
    
    t1new,dy1new,y_obs1new=t[t>=rangemin],dy[t>=rangemin],y_obs[t>=rangemin]
    t2new,dy2new,y_obs2new=t1new[t<=rangemax],dy1new[t<=rangemax],y_obs1new[t<=rangemax]   
    return t2new,dy2new,y_obs2new

def get_mean_interval(t):
    return int(np.max(t)-np.min(t))/(len(t)-1)

def get_interval(t):
    return int(np.max(t)-np.min(t))




def get_new_range_datas(m,n,dcf_lists,rangemin,rangemax):
    path1=dcf_lists[m]
    path2=dcf_lists[n] 
    data1=get_data(path1)
    t1,dy1,y_obs1=get_lc_data(data1)
    t1new,dy1new,y_obs1new=get_new_range_data(t1,dy1,y_obs1,rangemin,rangemax)
    data2=get_data(path2)
    t2,dy2,y_obs2=get_lc_data(data2)   
    t2new,dy2new,y_obs2new=get_new_range_data(t2,dy2,y_obs2,rangemin,rangemax) 
    return t1new,dy1new,y_obs1new,t2new,dy2new,y_obs2new


def get_dcf_name(m,n,dcf_lists,rangemin,rangemax,datadir):
    path1=dcf_lists[m]
    path2=dcf_lists[n]    
    newpath1=path1.split('/')[-1][4:-4]
    newpath2=path2.split('/')[-1][4:-4]
    newfile1='%s/dcf-%s-%s-%s-%s.csv'%(datadir,rangemin,rangemax,newpath1,newpath2)
    newfile2='%s/dcf-%s-%s-%s-%s.csv'%(datadir,rangemin,rangemax,newpath2,newpath1)  
    return newpath1,newpath2,newfile1,newfile2



def write_dcf_data(m,n,dcf_lists,rangemin,rangemax,datadir):     
    
    
    t1new,dy1new,y_obs1new,t2new,dy2new,y_obs2new=get_new_range_datas(m,n,dcf_lists,rangemin,rangemax)
    
    y1_norm=(y_obs1new-np.average(y_obs1new))/np.std(y_obs1new)
    y2_norm=(y_obs2new-np.average(y_obs2new))/np.std(y_obs2new)
    
    y1err_norm=dy1new/np.std(y_obs1new)
    y2err_norm=dy2new/np.std(y_obs2new)
    
    newpath1,newpath2,newfile1,newfile2=get_dcf_name(m,n,dcf_lists,rangemin,rangemax,datadir)
    
    dcf_data_write1=pd.DataFrame({'t':t1new,'flux':y1_norm,'err':y1err_norm},index=None,columns=['t','flux','err'])
    dcf_data_write2=pd.DataFrame({'t':t2new,'flux':y2_norm,'err':y2err_norm},index=None,columns=['t','flux','err'])
    
    dcf_data_write1=dcf_data_write1.sort_values('t')
    dcf_data_write2=dcf_data_write2.sort_values('t')
    
    dcf_data_write1.to_csv(newfile1,index=False,header=False)    
    dcf_data_write2.to_csv(newfile2,index=False,header=False)


# In[74]:


def get_time_lag_two_series_specific(m,n,dcf_lists,rangemin,rangemax,datadir,l_length,h_length,dt_width,w='gauss',p=2,):     
    
    newpath1,newpath2,newfile1,newfile2=get_dcf_name(m,n,dcf_lists,rangemin,rangemax,datadir)    
    t1new,dy1new,y_obs1new,t2new,dy2new,y_obs2new=get_new_range_datas(m,n,dcf_lists,rangemin,rangemax)
    
    print('mean interval: ',newpath1,get_mean_interval(t1new))        
    print('mean interval: ',newpath2,get_mean_interval(t2new))
    print('timerange:',rangemin,'-',rangemax)
    print('lag_range:',l_length,'-',h_length)
    print('dT:',dt_width)
    write_dcf_data(m,n,dcf_lists,rangemin,rangemax,datadir)   
    #(newfile1,newfile2)
    #dt_width=min(get_mean_interval(t1new),get_mean_interval(t2new))*10
    #lh_length=min(get_interval(t1new),get_interval(t2new))*0.5
    #print(get_mean_interval(t1new),get_mean_interval(t2new))
    
    command='python3 dcf_ngc1566.py %s %s %d %d %d %d %d -w=%s -p=%d -o '%(newfile1,newfile2,l_length,h_length,dt_width,rangemin,rangemax,w,p)
    print(command)
    os.system(command)


# In[75]:


def finish_specific(m,n,dcf_lists,rangemin,rangemax,l_length,h_length,dt_width):
    datadir='/Users/brettlv/timing_analysis/pydcf/ngc1566'
    os.chdir(datadir)
    for i in range(3):
        get_time_lag_two_series_specific(m,n,dcf_lists,rangemin,rangemax,datadir,l_length,h_length,dt_width,w='gauss',p=i)     
    for i in range(3):    
        get_time_lag_two_series_specific(m,n,dcf_lists,rangemin,rangemax,datadir,l_length,h_length,dt_width,w='slot',p=i) 

In [ ]:
pwd

In [ ]:
for i,j in enumerate(dcf_lists):
    print(i,j.split('/')[-1][4:-4])

In [ ]:
def plot_line_dcf(m,dcf_lists):
    dcf_path=dcf_lists[m]   
    dcf_data=pd.read_csv(dcf_path,header=None)
    ax.errorbar(dcf_data[0], dcf_data[1],dcf_data[2],fmt='.',label=dcf_path.split('/')[-1][4:-4])  
    
def plot_line_dcf_mn(m,n,dcf_lists):     
    plot_line_dcf(m,dcf_lists)    
    plot_line_dcf(n,dcf_lists)

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(8,5))
plot_line_dcf_mn(1,2,dcf_lists)
ax.legend()
time_range=overlap_timeinterval(m,n)
#rangemin,rangemax=(58250,58950)
rangemin,rangemax=(time_range[1],time_range[2])
ax.set_xlim(rangemin,rangemax)

## get overlap_timeinterval

In [ ]:
if True:
    m=3
    n=5
    path1=dcf_lists[m]
    path2=dcf_lists[n]    
    dcf_data_1=pd.read_csv(path1,header=None)
    dcf_data_2=pd.read_csv(path2,header=None)

In [ ]:
def overlap_timeinterval(m,n):
    path1=dcf_lists[m]
    path2=dcf_lists[n]    
    dcf_data_1=pd.read_csv(path1,header=None)
    dcf_data_2=pd.read_csv(path2,header=None)
    time_range=np.array([min(dcf_data_1[0]),max(dcf_data_1[0]),min(dcf_data_2[0]),max(dcf_data_2[0])])
    print(path1.split('/')[-1][4:-4],path2.split('/')[-1][4:-4])
    time_range.sort()
    return time_range

In [ ]:
m,n=(1,2)
time_range=overlap_timeinterval(m,n)
rangemin,rangemax=(time_range[1],time_range[2])
print(time_range)

## dcf_cal

In [ ]:
m,n=(2,1)
time_range=overlap_timeinterval(m,n)
rangemin,rangemax=(54000,59000)
#rangemin,rangemax=(time_range[1],time_range[2])
print(time_range)
finish_specific(m,n,dcf_lists,rangemin,rangemax,l_length=-500,h_length=500,dt_width=35)

In [ ]:
m,n=(2,1)
time_range=overlap_timeinterval(m,n)
rangemin,rangemax=(54000,59000)
#rangemin,rangemax=(time_range[1],time_range[2])
print(time_range)
finish_specific(m,n,dcf_lists,rangemin,rangemax,l_length=-200,h_length=200,dt_width=15)

## def plot

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(8,5))
plot_line_dcf_mn(1,20,dcf_lists)
ax.legend()

time_range=overlap_timeinterval(m,n)
rangemin,rangemax=(time_range[0],time_range[3])
#rangemin,rangemax=(58250,58950)
ax.set_xlim(rangemin,rangemax)
ax.semilogy()

In [ ]:
m,n=(1,20)
time_range=overlap_timeinterval(m,n)
rangemin,rangemax=(time_range[1],time_range[2])
print(time_range)
finish_specific(m,n,dcf_lists,rangemin,rangemax,l_length=-500,h_length=500,dt_width=25)

# ccf

In [ ]:
def get_ccf_name(m,n,ccf_lists,rangemin,rangemax,datadir):
    path1=ccf_lists[m]
    path2=ccf_lists[n]    
    newpath1=path1.split('/')[-1][4:-4]
    newpath2=path2.split('/')[-1][4:-4]
    newfile1='%s/ccf-%s-%s-%s-%s.csv'%(datadir,rangemin,rangemax,newpath1,newpath2)
    newfile2='%s/ccf-%s-%s-%s-%s.csv'%(datadir,rangemin,rangemax,newpath2,newpath1)  
    return newpath1,newpath2,newfile1,newfile2    
    
def write_ccf_data(m,n,ccf_lists,rangemin,rangemax,datadir):     
    
    
    t1new,dy1new,y_obs1new,t2new,dy2new,y_obs2new=get_new_range_datas(m,n,ccf_lists,rangemin,rangemax)
    
    y1_norm=(y_obs1new-np.average(y_obs1new))/np.std(y_obs1new)
    y2_norm=(y_obs2new-np.average(y_obs2new))/np.std(y_obs2new)
    
    y1err_norm=dy1new/np.std(y_obs1new)
    y2err_norm=dy2new/np.std(y_obs2new)
    
    newpath1,newpath2,newfile1,newfile2=get_ccf_name(m,n,ccf_lists,rangemin,rangemax,datadir)
    
    ccf_data_write1=pd.DataFrame({'t':t1new,'flux':y1_norm,'err':y1err_norm},index=None,columns=['t','flux','err'])    
    ccf_data_write2=pd.DataFrame({'t':t2new,'flux':y2_norm,'err':y2err_norm},index=None,columns=['t','flux','err'])
    
    ccf_data_write1=ccf_data_write1.sort_values('t')
    ccf_data_write2=ccf_data_write2.sort_values('t')
    
    ccf_data_write1.to_csv(newfile1,index=False,header=False)
    ccf_data_write2.to_csv(newfile2,index=False,header=False) 
    
def get_ccf(m,n,dcf_lists,rangemin,rangemax,datadir):
    newpath1,newpath2,newfile1,newfile2=get_ccf_name(m,n,dcf_lists,rangemin,rangemax,datadir)    
    t1new,dy1new,y_obs1new,t2new,dy2new,y_obs2new=get_new_range_datas(m,n,dcf_lists,rangemin,rangemax)
    write_ccf_data(m,n,dcf_lists,rangemin,rangemax,datadir)               
    command='python2 ccf_ngc1566_cpython.py %s %s %d %d'%(newfile1,newfile2,rangemin,rangemax,)
    print(command)
    #os.system(command)    

In [ ]:
dataccf='/Users/brettlv/timing_analysis/python_ccf_code/Release/CPython/ngc1566_ccf'
os.chdir(dataccf)
m,n=(2,1)
time_range=overlap_timeinterval(m,n)
rangemin,rangemax=(54000,59000)
#rangemin,rangemax=(time_range[1],time_range[2])
print(time_range)
get_ccf(1,2,dcf_lists,rangemin,rangemax,datadir=dataccf)

# gauss_fit

https://docs.astropy.org/en/stable/modeling/fitting.html#simple-1-d-model-fitting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from scipy.optimize import curve_fit
# Generate fake data
np.random.seed(0)

# Fit the data using a Gaussian

x=data_xrt_fit_NGC1566['obs_time']
y=data_xrt_fit_NGC1566['xflux']*1.0e12
x,y=select_range(x,y,56000,58600)


param_bounds=([-np.inf,57500,0],[np.inf,58500,np.inf])
popt, pcov = curve_fit(gauss_function, x, y, p0 = [100., 58000, 100],bounds=param_bounds)

print(popt)

g_init = models.Gaussian1D(amplitude=100., mean=58200, stddev=500.)
fit_g = fitting.LevMarLSQFitter()
g = fit_g(g_init, x, y)

# Plot the data with the best-fit model
plt.figure(figsize=(8,5))
plt.plot(x, y, 'ko')


x_new=np.linspace(min(x),max(x))
plt.scatter(x_new, g(x_new), label='Gaussian')
plt.scatter(x_new, gauss_function(x_new, *popt), label='curve_fit')

plt.xlabel('mjd')
plt.ylabel('Flux')
plt.legend(loc=2)
print(g)

In [ ]:
def select_range(x,y,Tmin,Tmax):
    x_=[]
    y_=[]
    for i,j in zip(x,y):
        if i>=Tmin and i<=Tmax:
            x_.append(i)
            y_.append(j)
    return  np.array(x_),np.array(y_)       
    

In [ ]:
def gauss_function(x, a, x0, sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))


In [ ]:
##### import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting

# Generate fake data
np.random.seed(0)

# Fit the data using a Gaussian


x=data_ASAS_NGC1566['hmjd']
y=data_ASAS_NGC1566['flux (mJy)']

x,y=select_range(x,y,58180,59000)



g_init = models.Gaussian1D(amplitude=80., mean=58200, stddev=100.)
fit_g = fitting.LevMarLSQFitter()
g = fit_g(g_init, x, y)

# Plot the data with the best-fit model
plt.figure(figsize=(8,5))
plt.plot(x, y, 'ko')
x_new=np.linspace(min(x),max(x))
plt.scatter(x_new, g(x_new), label='Gaussian')

plt.xlabel('mjd')
plt.ylabel('Flux')
plt.legend(loc=2)
print(g)

In [ ]:
58324-58280

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting

# Generate fake data
np.random.seed(0)

# Fit the data using a Gaussian


for i in range(1,2):
    name_mjd='mjd'
    name_band=data_neowise_select.columns[i*2]
    name_err=data_neowise_select.columns[i*2+1]
    print(name_mjd,name_band,name_err)
    mag_neowise_data=pd.DataFrame({'t':data_neowise[name_mjd],'flux':data_neowise[name_band],'err':data_neowise[name_err]},index=None,columns=['t','flux','err'])
    dcf_neowise_data=pd.DataFrame({'t':data_neowise[name_mjd],'flux':(-48.6-data_neowise[name_band])/2.5,'err':data_neowise[name_err]/2.5},index=None,columns=['t','flux','err'])    

x=dcf_neowise_data['t']
y=dcf_neowise_data['flux']

x,y=select_range(x,y,57600,59000)

g_init = models.Gaussian1D(amplitude=-20., mean=58323, stddev=100.)
fit_g = fitting.LevMarLSQFitter()
g = fit_g(g_init, x, y)



    
    
# Plot the data with the best-fit model
plt.figure(figsize=(8,5))
plt.plot(x, y, 'ko')

x_new=np.linspace(min(x),max(x))
plt.scatter(x_new, g(x_new), label='Gaussian')


plt.xlabel('mjd')
plt.ylabel('Flux')
plt.legend(loc=2)
print(g)

# [SED](./NGC1566_sed.ipynb)

In [ ]:
sed_ASAS_data=pd.DataFrame({'t':data_ASAS_NGC1566['hmjd'],'flux':data_ASAS_NGC1566['flux (mJy)'],'err':data_ASAS_NGC1566['flux err']},index=None,columns=['t','flux','err'])    